In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import torch

from csv import QUOTE_NONE
from datetime import datetime
from pprint import pprint


if 'src' not in sys.path:
    sys.path.append('src')

%reload_ext autoreload
%autoreload 2

from explore.data import DetectorDataProvider, LookUpTable
from explore.graph import IntersectionGraph

Construct a correlation coefficient matrix
$$
R(i, j)=\frac{\sum_{k=1}^d(z(s_i,t_k)-\tilde{z}(s_i))(z(s_j,t_k)-\tilde{z}(s_j))}{\sqrt{\sum_{k=1}^d(z(s_i,t_k)-\tilde{z}(s_i))^2}\sqrt{\sum_{k=1}^d(z(s_j,t_k)-\tilde{z}(s_j))^2}},
$$
where $$\tilde{z}(s_i)=\frac{1}{d}\sum_{k=1}^dz(s_i,t_k)$$

In [ ]:
def construct_r(mat_q, ignore: set=None):
    if ignore is None:
        ignore = set()
    d, p = mat_q.shape
    mat_q_normalized = mat_q - mat_q.mean(dim=0)
    mat_r = torch.zeros(p, p)
    for i in range(p):
        for j in range(i, p):
            if (i, j) in ignore:
                mat_r[i][j] = 0.0
            else:
                i_col = mat_q_normalized[:, i]
                j_col = mat_q_normalized[:, j]
                i_norm = max(torch.norm(i_col), 1e-12)
                j_norm = max(torch.norm(j_col), 1e-12)
                if i_norm == 0 or j_norm == 0:
                    if i_norm == j_norm:
                        mat_r[i][j] = 1
                    else:
                        mat_r[i][j] = 0
                else:
                    mat_r[i][j] = (i_col @ j_col) / i_norm / j_norm
            mat_r[j][i] = mat_r[i][j]
    return mat_r

mat_r = construct_r(mat_q)
torch.save(mat_r, 'mat_r_excluded_missing.pt')
mat_r

Load matrices R and Q

In [ ]:
mat_q = torch.load('mat_q_excluded_missing.pt')
mat_r = torch.load('mat_r_excluded_missing.pt')
mat_r = mat_r / mat_r.max()

In [ ]:
ALPHA = 0.9999

n_timesteps, n_sections = mat_q.shape
n_grouped = 0
groups = []

mat_r_copy = mat_r - torch.diag(mat_r.diag())
while n_grouped < n_sections:
    new_group_idx = torch.nonzero(mat_r_copy > ALPHA)
    if len(new_group_idx) > 0:
        corr = mat_r[new_group_idx[:, 0], new_group_idx[:, 1]]
        new_group_idx = new_group_idx[:, 0].unique()

        n_grouped += len(new_group_idx)
        mat_r_copy[new_group_idx, :] = 0
        mat_r_copy[:, new_group_idx] = 0
        groups.append((new_group_idx, corr.min(), corr.max()))
        if mat_r_copy.max() == 0:
            break
        else:
            mat_r_copy /= mat_r_copy.max()
del mat_r_copy

Analyse grouping

In [ ]:
print(f'Using alpha={ALPHA}, {n_sections} correlated sections were divided ' +
      f'into {len(groups)} groups:')
n_ungrouped = 0
for i, (group, corr_min, corr_max) in enumerate(groups, start=1):
      group_coeffs = mat_r[tuple(group.T), :]
      print(f'Group {i} - {len(group)} sections with correlation coefficients '
          f'{corr_min:.3f} to {corr_max:.3f}')
print(str(n_sections - n_grouped) + " section(s) don't correlate with anything and weren't grouped")


In [ ]:
representatives = torch.stack([g[0] for g, _, _ in groups])
mat_c = mat_q[:, representatives]
assert mat_c.shape == (mat_q.shape[0], len(groups))
torch.save(mat_c, 'mat_c_excluded_missing.pt')
mat_c.shape

In [ ]:
mat_x = torch.linalg.pinv(mat_c) @ mat_q
torch.save(mat_x, 'mat_x_excluded_missing.pt')
mat_x.shape

In [ ]:
print(torch.max((mat_c @ mat_x) - mat_q))